#  Procedure to identify the sources and get FWHM (gaussian) for aperture determination.

To perform aperture photometry, we need to determine apertures and before that mark the sources we are interested in. The algorithms usually give 1000+ sources in a single frame based on our threshold. Performing photometry on these detected sources is a waste of time as well as computing power. Thus we need to mark the source and a few comparision stars depending on our requirements.


We use the PYTHON package GINGA to estimate the location (xcentroid, ycentroid) of the sources and then calculate the FWHM for the Gaussian/Moffat fits to the profiles in order to estimate the required aperture size for photometric calculation. 

In [1]:
import ginga
from ginga.web.pgw import ipg
import numpy as np
from ginga.util import iqcalc

Now we need a display environment for interactive inputs. We have plenty of options for this purpose, but we choose a localhost so that we can display the image in the web browser itself. The browser should support html5. No need to enter any other window. It will also save a lot of pointing and clicking!!!

We need to make sure the port we will choose is not occupied yet otherwise a port busy error will be there.


In [2]:
use_opencv = False
server = ipg.make_server(host='localhost', port=8711, use_opencv=use_opencv)
server.start(no_ioloop=True)

Now the server is started at localhost:xxxx. We will insert a viewer onto that and then we will load a fits file in that viewer. This image can be any of the science frames we are analysing, but for best results we should choose the best image we can make out of the multiple images intuitively ( best seeing conditions, dark night etc.)

In [3]:
tv = server.get_viewer('tv')
tv.load('test.fits')
tv.open()


Sometimes the viewer will not open by default. So, we get the address manually and insert it in the browser. It will display it then.

In [4]:
tv.url

'http://localhost:8711/app?id=tv'

In [5]:
tv.embed(height=600)

To work interactively with the image viewer, we add a canvas to the image and perform the operations subsequently.

In [6]:
canvas = tv.add_canvas()
canvas.delete_all_objects()
# set the drawing parameters


In [7]:
canvas = tv.add_canvas()
canvas.set_drawtype('point',size=4, color='black')

In [8]:
for i in range(len(canvas.objects)):
    k= canvas.objects[i]
    print(k.x,k.y)



1603.9833333333333 624.835
1689.275 720.3616666666667
1184.3483333333334 607.7766666666666
1204.8183333333334 713.5383333333334


In [23]:
canvas.delete_all_objects()

Now we have obtained the positions for the source as well as the reference stars in the frame.
Next  step is to select the rectangular region to  get the mean FWHM of the sources detected in the selected region.

It will give the total number of sources within the selected region.

In [60]:
canvas=tv.add_canvas()
canvas.delete_all_objects()
canvas.set_drawtype('square')


In [61]:
iq = iqcalc.IQCalc()
img = tv.get_image()
r = canvas.objects[0]
data = img.cutout_shape(r)
peaks = iq.find_bright_peaks(data)
len(peaks)
obj=iq.evaluate_peaks(peaks,data)

/home/vivek/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:746: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [64]:
len(peaks)


99

Now we will calculate the FWHM of detected sources from the selected region.

In [63]:
fw=np.zeros(len(obj))
for i in range(0,len(obj)):
    fw[i]=obj[i]['fwhm']
    
print('Median FWHM',np.median(fw),0.53*np.median(fw))
print('Mean FWHM',np.mean(fw),0.53*np.mean(fw))

Median FWHM 7.307715506613564 3.8730892185051893
Mean FWHM 8.411168611877594 4.457919364295125


In [18]:
np.mean(fw)

7.880906173404611

In [1]:
from ginga.web.jupyterw.ImageViewJpw import EnhancedCanvasView

# Ginga viewer for Jupyter notebook.
v1 = EnhancedCanvasView(logger=logger)

# Set our linkage between the Jupyter widget at Ginga viewer.
v1.set_widget(jup_img)

# Enable all possible keyboard and pointer operations.
bd = v1.get_bindings()
bd.enable_all(True)

ModuleNotFoundError: No module named 'ipyevents'